In [74]:
from torch import nn     
import torch
import sys
import numpy as np
import sys
sys.path.append("/home3/ebrahim2/neural_seq_decoder/src/neural_decoder/")
from bit import Transformer, BiT_Phoneme, pad_to_multiple, create_temporal_mask, pair
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from augmentations import GaussianSmoothing
from dataset import pad_to_multiple
import copy 
from torchtnt.utils.flops import FlopTensorDispatchMode

In [26]:
neural_dim = 256
kernel_len = 32
layer_dim = 5
hidden_dim = 1024

In [124]:
gru_decoder = nn.GRU(
            (neural_dim) * kernel_len,
            hidden_dim,
            layer_dim,
            batch_first=True,
            dropout=0,
            bidirectional=False
        )

fc_decoder_out = nn.Linear(hidden_dim, 40 + 1) 

fc_decoder_out_2 = nn.Linear(384, 40 + 1) 

In [127]:
inputs = torch.randn(1,500,256) # 10 seconds of input

In [128]:
daySpecific = nn.Linear(256, 256)
with FlopTensorDispatchMode(daySpecific) as ftdm:
    res = daySpecific(inputs)
    flops_forward = copy.deepcopy(ftdm.flop_counts)

    
total_flops = sum(                # outer sum
    sum(inner.values())           #  ← inner sum
    for inner in flops_forward.values()
)

print(f"Total forward FLOPs: {total_flops:,}")        # e.g. 53,502,976
print(f"≈ {total_flops/1e6/10:.2f} MFLOPs")              # or /1e9 for GFLOPs

Total forward FLOPs: 32,768,000
≈ 3.28 MFLOPs


In [129]:
unfolder = torch.nn.Unfold(
            (kernel_len, 1), dilation=1, padding=0, stride=4
        )
stridedInputs = torch.permute(
            unfolder(
                torch.unsqueeze(torch.permute(inputs, (0, 2, 1)), 3)
            ),
            (0, 2, 1),
        )
stridedInputs.shape

torch.Size([1, 118, 8192])

In [131]:
32*20 + (4*20)*117

10000

In [134]:
import copy 
from torchtnt.utils.flops import FlopTensorDispatchMode
stridedInputs = torch.randn(1,118, neural_dim*kernel_len)
with FlopTensorDispatchMode(gru_decoder) as ftdm:
    res_gru, _ = gru_decoder(stridedInputs)
    flops_forward = copy.deepcopy(ftdm.flop_counts)

    
total_flops = sum(                # outer sum
    sum(inner.values())           #  ← inner sum
    for inner in flops_forward.values()
)

print(f"Total forward FLOPs: {total_flops:,}")        # e.g. 53,502,976
print(f"≈ {total_flops/1e6/10:.2f} MFLOPs")              # or /1e9 for GFLOPs

Total forward FLOPs: 6,310,330,368
≈ 631.03 MFLOPs


In [135]:
with FlopTensorDispatchMode(fc_decoder_out) as ftdm:
    res2 = fc_decoder_out(res_gru)
    flops_forward = copy.deepcopy(ftdm.flop_counts)

    
total_flops = sum(                # outer sum
    sum(inner.values())           #  ← inner sum
    for inner in flops_forward.values()
)

print(f"Total forward FLOPs: {total_flops:,}")        # e.g. 53,502,976
print(f"≈ {total_flops/1e6/10:.2f} MFLOPs")              # or /1e9 for GFLOPs

Total forward FLOPs: 4,954,112
≈ 0.50 MFLOPs


In [136]:
631.03 + 0.50 + 3.28

634.81

In [138]:
import copy 
from torchtnt.utils.flops import FlopTensorDispatchMode
tf_inputs = torch.randn(1,500,256*5)
patch_transform = nn.Linear(256*5, 384)
with FlopTensorDispatchMode(patch_transform) as ftdm:
    _ = patch_transform(tf_inputs)
    #seq_out = fc_decoder_out_2(res)
    flops_forward = copy.deepcopy(ftdm.flop_counts)
    
total_flops = sum(                # outer sum
    sum(inner.values())           #  ← inner sum
    for inner in flops_forward.values()
)

print(f"Total forward FLOPs: {total_flops:,}")        # e.g. 53,502,976
print(f"≈ {total_flops/1e6/10:.2f} MFLOPs")              # or /1e9 for GFLOPs


Total forward FLOPs: 245,760,000
≈ 24.58 MFLOPs


In [119]:
model = Transformer(384, 7, 6, 64, 4, 
                                    0, use_relative_bias=True)

import copy 
from torchtnt.utils.flops import FlopTensorDispatchMode
tf_inputs = torch.randn(1,100,384)
with FlopTensorDispatchMode(model) as ftdm:
    res = model(tf_inputs)
    #seq_out = fc_decoder_out_2(res)
    flops_forward = copy.deepcopy(ftdm.flop_counts)
    
total_flops = sum(                # outer sum
    sum(inner.values())           #  ← inner sum
    for inner in flops_forward.values()
)

print(f"Total forward FLOPs: {total_flops:,}")        # e.g. 53,502,976
print(f"≈ {total_flops/1e6/10:.2f} MFLOPs")              # or /1e9 for GFLOPs


Total forward FLOPs: 1,875,148,800
≈ 187.51 MFLOPs


In [120]:
with FlopTensorDispatchMode(fc_decoder_out_2) as ftdm:
    res2 = fc_decoder_out_2(res)
    #seq_out = fc_decoder_out(res)
    flops_forward = copy.deepcopy(ftdm.flop_counts)

    
total_flops = sum(                # outer sum
    sum(inner.values())           #  ← inner sum
    for inner in flops_forward.values()
)

print(f"Total forward FLOPs: {total_flops:,}")        # e.g. 53,502,976
print(f"≈ {total_flops/1e6/10:.2f} MFLOPs")              # or /1e9 for GFLOPs

Total forward FLOPs: 629,760
≈ 0.06 MFLOPs


In [121]:
187.51 + 0.06 + 9.83

197.4